<a href="https://colab.research.google.com/github/Azure06072005/DS317---Data-Mining-for-Enterprise/blob/main/w4_nhom2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tính toán mức độ tương tác của người dùng

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark -q
!sudo apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
# Define filepath
BASE_DIR = "/content/drive/MyDrive/Dataset"
!tree {BASE_DIR}

/content/drive/MyDrive/Dataset
├── course_proportion.csv
├── data_for_Score_Struct.csv
├── entities
│   ├── comment.json
│   ├── concept.json
│   ├── course.json
│   ├── other.json
│   ├── paper.json
│   ├── problem.json
│   ├── reply.json
│   ├── school.json
│   ├── teacher.json
│   ├── user.json
│   └── video.json
├── prerequisites
│   ├── cs.json
│   └── psy.json
├── relations
│   ├── comment-reply.txt
│   ├── concept-course.txt
│   ├── course-comment.txt
│   ├── course-field.json
│   ├── course-school.txt
│   ├── course-teacher.txt
│   ├── exercise-problem.txt
│   ├── MOOCCubeX -> /content/drive/.shortcut-targets-by-id/10jSoO03qqTeW5GxqZxGTcsu4obB8Dg7X/MOOCCubeX
│   ├── MOOCCubeX (1) -> /content/drive/.shortcut-targets-by-id/10jSoO03qqTeW5GxqZxGTcsu4obB8Dg7X/MOOCCubeX
│   ├── reply-reply.txt
│   ├── user-problem.json
│   ├── user-problem-out
│   ├── user-problem-split
│   │   ├── part-00000-1ffbdd59-9b66-4d86-9346-5bce8141fa4b-c000.snappy.parquet
│   │   ├── part-00001-1ffbdd59-9b6

In [ ]:
!ls {BASE_DIR}

course_proportion.csv		  resource_video_course.parquet
data_for_Score_Struct.csv	  user_id_course_df.csv
entities			  user_problem_res2.parquet
prerequisites			  user_problem_res3.parquet
relations			  user_problem_res.parquet
resource_exercise_course.parquet  users-course-labeling.csv


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round
spark = SparkSession.builder \
  .appName("NySparkApplication") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

# Hiện DataFrame

## Analyze speed

### Subtask:
Analyze the distribution of `speed` values to understand how users adjust playback speed.

**Reasoning**:
Calculate descriptive statistics for the `speed` column in the `df_flattened` DataFrame to understand the distribution of playback speeds.

In [ ]:
df_user_video = spark.read.parquet(BASE_DIR + "/relations/user-video/user-video-seg*.parquet")
df_user_video.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Tách dữ liệu từ cột seq

In [ ]:
from os import truncate

# Explode the outer array in 'seq' to get each video sequence as a row
df_exploded = df_user_video.select("user_id", explode("seq").alias("video_sequence"))

# Select the video_id and explode the 'segment' array within each video sequence
df_exploded = df_exploded.select(
    "user_id",
    df_exploded["video_sequence"]["video_id"].alias("video_id"),
    explode(df_exploded["video_sequence"]["segment"]).alias("segment_details")
)

# Select the individual fields from the 'segment_details' struct
df_flattened = df_exploded.select(
    "user_id",
    "video_id",
    df_exploded["segment_details"]["start_point"].alias("start_point"),
    df_exploded["segment_details"]["end_point"].alias("end_point"),
    df_exploded["segment_details"]["speed"].alias("speed"),
    df_exploded["segment_details"]["local_start_time"].alias("local_start_time")
    )

# Show the resulting DataFrame with separated columns
df_flattened.show(truncate=False)

+---------+---------+-----------+---------+-----+----------------+
|user_id  |video_id |start_point|end_point|speed|local_start_time|
+---------+---------+-----------+---------+-----+----------------+
|U_7329543|V_1404465|4.8        |164.9    |1.0  |1598177594      |
|U_7329543|V_1404469|572.4      |587.4    |1.0  |1598181080      |
|U_7329543|V_1404470|34.8       |54.8     |1.0  |1598181866      |
|U_7329543|V_1404470|52.2       |55.9     |1.0  |1598181962      |
|U_7329543|V_1404471|4.8        |182.5    |1.0  |1598181976      |
|U_7329543|V_1404472|4.8        |34.9     |1.0  |1598182171      |
|U_7329543|V_1404472|64.9       |99.5     |1.0  |1598182231      |
|U_7329543|V_1404473|6.9        |90.3     |2.0  |1598182329      |
|U_7329543|V_1404493|4.8        |44.9     |1.0  |1598265546      |
|U_7329543|V_1404493|75.9       |84.3     |1.0  |1598265880      |
|U_7329543|V_1404494|4.8        |19.9     |1.0  |1598265901      |
|U_7329543|V_1404494|18.4       |43.4     |1.0  |1598265938   

## User Problem

In [ ]:
df_user_problem = spark.read.parquet(BASE_DIR + "/relations/user-problem-split/part-000**-1ffbdd59-9b66-4d86-9346-5bce8141fa4b-c000.snappy.parquet")
df_user_problem.show(truncate=False)

+--------+----------+----------------+----------+-----+-------------------+----------+
|attempts|is_correct|log_id          |problem_id|score|submit_time        |user_id   |
+--------+----------+----------------+----------+-----+-------------------+----------+
|1       |1         |10091134_6899951|Pm_6899951|1.0  |2020-09-08 10:08:07|U_10091134|
|1       |0         |10209699_1676089|Pm_1676089|0.0  |2020-07-24 21:02:23|U_10209699|
|3       |0         |10359753_6565795|Pm_6565795|NULL |2020-10-26 00:58:00|U_10359753|
|1       |1         |10359421_8434194|Pm_8434194|NULL |2020-12-02 19:35:14|U_10359421|
|3       |1         |10020095_3919056|Pm_3919056|50.0 |2020-07-04 23:33:03|U_10020095|
|2       |1         |10359704_6565656|Pm_6565656|NULL |2020-12-02 22:49:37|U_10359704|
|1       |1         |10348419_8474076|Pm_8474076|NULL |2020-11-27 19:22:25|U_10348419|
|1       |1         |10460560_3295864|Pm_3295864|1.0  |2020-06-14 18:55:43|U_10460560|
|1       |1         |10383158_5596900|Pm_55

## Video

In [ ]:
df_video = spark.read.json(BASE_DIR + "/entities/video.json")
df_video.show(truncate=True)

+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|                ccid|                 end|                  name|               start|                                text|
+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|0001603F826A3DDA9...|[4.255, 8.119, 10...|                 Video|[1.031, 7.095, 8....|[第二个就是短助记符在生成上面。有...|
|0003DB14A14A53279...|[12.581, 17.38, 1...|                 Video|[11.799, 12.581, ...|  [大家好, 今天我们讨论一维系统辐...|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|        家禽的消化系统|[19.2, 20.84, 23....|   [各位同学 大家好, 我是来自中国...|
|00059EBD1371A6959...|[23.14, 194.458, ...|    1.2 计算机硬件系统|[19.372, 190.522,...| [大家好 本视频介绍计算机硬件系统...|
|0005D1DC01B4EF8F9...|[25.08, 26.775, 2...|      什么是供应链网络|[24.33, 25.081, 2...|   [大家好, 今天我们学习的知识点,...|
|0007919ED7652A779...|[7.26, 9.97, 11.5...|    1.2 英语文字的起源|[6.29, 7.31, 10.0...|  [同学们好, 今天我们

# Task
Analyze the user interaction data in `df_flattened`.

## Calculate video watch duration

### Subtask:
Calculate the duration of each video segment by subtracting `start_point` from `end_point`.


**Reasoning**:
Calculate the duration of each video segment by subtracting `start_point` from `end_point` and add it as a new column named `duration`.



In [ ]:
df_flattened = df_flattened.withColumn("duration", round(col("end_point") - col("start_point")))

df_flattened.show(truncate=False)

+---------+---------+-----------+---------+-----+----------------+--------+
|user_id  |video_id |start_point|end_point|speed|local_start_time|duration|
+---------+---------+-----------+---------+-----+----------------+--------+
|U_7329543|V_1404465|4.8        |164.9    |1.0  |1598177594      |160.0   |
|U_7329543|V_1404469|572.4      |587.4    |1.0  |1598181080      |15.0    |
|U_7329543|V_1404470|34.8       |54.8     |1.0  |1598181866      |20.0    |
|U_7329543|V_1404470|52.2       |55.9     |1.0  |1598181962      |4.0     |
|U_7329543|V_1404471|4.8        |182.5    |1.0  |1598181976      |178.0   |
|U_7329543|V_1404472|4.8        |34.9     |1.0  |1598182171      |30.0    |
|U_7329543|V_1404472|64.9       |99.5     |1.0  |1598182231      |35.0    |
|U_7329543|V_1404473|6.9        |90.3     |2.0  |1598182329      |83.0    |
|U_7329543|V_1404493|4.8        |44.9     |1.0  |1598265546      |40.0    |
|U_7329543|V_1404493|75.9       |84.3     |1.0  |1598265880      |8.0     |
|U_7329543|V

# Task
Analyze user interaction based on the `df_flattened` dataframe using EDA and visualizations.

## Analyze start points

### Subtask:
Analyze the distribution of `start_point` values to understand where users typically start watching videos.

## Analyze local start time

### Subtask:
Analyze the `local_start_time` to understand the time of day or week users are most active and visualize the results.

**Reasoning**:
Convert the `local_start_time` to a timestamp and extract the hour and day of the week to analyze user activity patterns.

In [ ]:
from pyspark.sql.functions import from_unixtime, hour, dayofweek

df_flattened = df_flattened.withColumn("timestamp", from_unixtime("local_start_time"))
df_flattened = df_flattened.withColumn("hour_of_day", hour("timestamp"))
df_flattened = df_flattened.withColumn("day_of_week", dayofweek("timestamp"))

df_flattened.select("local_start_time", "timestamp", "hour_of_day", "day_of_week").show(truncate=False)

+----------------+-------------------+-----------+-----------+
|local_start_time|timestamp          |hour_of_day|day_of_week|
+----------------+-------------------+-----------+-----------+
|1598177594      |2020-08-23 10:13:14|10         |1          |
|1598181080      |2020-08-23 11:11:20|11         |1          |
|1598181866      |2020-08-23 11:24:26|11         |1          |
|1598181962      |2020-08-23 11:26:02|11         |1          |
|1598181976      |2020-08-23 11:26:16|11         |1          |
|1598182171      |2020-08-23 11:29:31|11         |1          |
|1598182231      |2020-08-23 11:30:31|11         |1          |
|1598182329      |2020-08-23 11:32:09|11         |1          |
|1598265546      |2020-08-24 10:39:06|10         |2          |
|1598265880      |2020-08-24 10:44:40|10         |2          |
|1598265901      |2020-08-24 10:45:01|10         |2          |
|1598265938      |2020-08-24 10:45:38|10         |2          |
|1598266072      |2020-08-24 10:47:52|10         |2    

**Reasoning**:
The error indicates a connection issue with Spark. I will restart the Spark session and reload the dataframes before attempting the local start time analysis again.

In [ ]:
spark.stop()
spark = SparkSession.builder \
  .appName("NySparkApplication") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

df_user_video = spark.read.parquet(BASE_DIR + "/relations/user-video/user-video-seg*.parquet")

from pyspark.sql.functions import explode, col, round, sum

df_exploded = df_user_video.select("user_id", explode("seq").alias("video_sequence"))

df_exploded = df_exploded.select(
    "user_id",
    df_exploded["video_sequence"]["video_id"].alias("video_id"),
    explode(df_exploded["video_sequence"]["segment"]).alias("segment_details")
)

df_flattened = df_exploded.select(
    "user_id",
    "video_id",
    df_exploded["segment_details"]["start_point"].alias("start_point"),
    df_exploded["segment_details"]["end_point"].alias("end_point"),
    df_exploded["segment_details"]["speed"].alias("speed"),
    df_exploded["segment_details"]["local_start_time"].alias("local_start_time")
)

df_flattened = df_flattened.withColumn("duration", round(col("end_point") - col("start_point")))
df_video_watch_time = df_flattened.groupBy("user_id", "video_id").agg(sum("duration").alias("total_watch_time"))

df_video_watch_time.show(truncate=False)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <exception str() failed>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^

Py4JError: py4j does not exist in the JVM

**Reasoning**:
Calculate descriptive statistics for the `start_point` column in the `df_flattened` DataFrame to understand the distribution of start points.

In [ ]:
descriptive_stats_start_point = df_flattened.select("start_point").summary("count", "mean", "stddev", "min", "max")
descriptive_stats_start_point.show()

# Task
Analyze the user interaction data in the provided dataframe, including EDA, data transformation, normalization, and cleaning. Visualize the findings with graphs and summarize the key insights.

## Analyze end points

### Subtask:
Analyze the distribution of `end_point` values to understand where users typically stop watching videos.


**Reasoning**:
Calculate descriptive statistics for the `end_point` column in the `df_flattened` DataFrame to understand the distribution of end points.



In [ ]:
descriptive_stats_end_point = df_flattened.select("end_point").summary("count", "mean", "stddev", "min", "max")
descriptive_stats_end_point.show()

## Calculate total watch time per user and video

### Subtask:
Calculate the total watch duration for each user and video combination.

**Reasoning**:
Group the `df_flattened` DataFrame by `user_id` and `video_id` and calculate the sum of `duration` for each group to get the total watch time per user and video.

In [ ]:
from pyspark.sql.functions import sum, round, col

df_flattened = df_flattened.withColumn("duration", round(col("end_point") - col("start_point")))
df_video_watch_time = df_flattened.groupBy("user_id", "video_id").agg(sum("duration").alias("total_watch_time"))

df_video_watch_time.show(truncate=False)